In [1]:
using Distributions
using Optim

In [2]:
function generate_data(N)
    delta1, delta2 = 3 , -2
    beta1, beta2 = [1 4], [2 3]
    rho = -2
    lambda = exp(rho)/(1+exp(rho))

    Y = zeros(N,2)
    X = zeros(N,2)

    for i=1:N
        x1,x2 = rand(Normal(0.0, 1.0),3)
        e1,e2 = rand(Logistic(),2)
        select = rand(Categorical([lambda, (1-lambda)]))

        #see logit response i.e. definitely 11
        eq = []
        for y_init=[[0 0],[1 1],[0 1],[1 0]]
            y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0

            if true
                println( "$(1*y1) $(1*y2),$(y_init)" )
                println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
                println("-----")
            end

            if (y1==y_init[1]) & (y2==y_init[2])
                append!(eq,[y_init])
            end
        end
        
        y1, y2 = eq[1]
        if size(eq)[1] >= 2
            y1, y2 = eq[select]
        end
        Y[i,:] =[y1 y2]
        X[i,:] =[x1 x2]
    end
    return Y,X
end

Y,X = generate_data(200)

1 1,[0 0]
5.177318781689965 3.4095719154521937
-----
1 1,[1 1]
8.177318781689966 1.4095719154521935
-----
1 1,[0 1]
8.177318781689966 3.4095719154521937
-----
1 1,[1 0]
5.177318781689965 1.4095719154521935
-----
1 1,[0 0]
2.4861278958927513 3.8514732562526066
-----
1 1,[1 1]
5.486127895892751 1.8514732562526066
-----
1 1,[0 1]
5.486127895892751 3.8514732562526066
-----
1 1,[1 0]
2.4861278958927513 1.8514732562526066
-----
0 1,[0 0]
-3.0011739421116213 3.201450598421754
-----
0 1,[1 1]
-0.0011739421116212512 1.2014505984217538
-----
0 1,[0 1]
-0.0011739421116212512 3.201450598421754
-----
0 1,[1 0]
-3.0011739421116213 1.2014505984217538
-----
0 0,[0 0]
-4.716191175324796 -0.6080154132822213
-----
0 0,[1 1]
-1.716191175324796 -2.6080154132822213
-----
0 0,[0 1]
-1.716191175324796 -0.6080154132822213
-----
0 0,[1 0]
-4.716191175324796 -2.6080154132822213
-----
1 0,[0 0]
5.535235623329365 -0.39544693962463184
-----
1 0,[1 1]
8.535235623329365 -2.395446939624632
-----
1 0,[0 1]
8.5352356233

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

In [11]:
function ll_obj(params;xs=0,ys=0)
    n = size(X)[1]
    k = Integer(size(X)[2]/2) + 1
    p = []
    
    for bound=[[0,0],[1,1],[0,1],[1,0]]
        util1 = params[1] .+ xs[:,1].*params[2]  .+ bound[2].*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ bound[1].*params[6]
        prob = exp.(util1).^(bound[1])./(1 .+ exp.(util1))
        prob = prob .* exp.(util2).^(bound[2])./(1 .+ exp.(util2))
        append!(p,[prob])
    end
    
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    #lambda = exp(params[7])/(1+exp(params[7]))
    lambda = 1
    
    ll = p00 .* p[1] + p11 .*p[2]
    ll = ll .+ p01 .* ( lambda .* p[3] + (1-lambda).*(1 .- p[1] .- p[2] .- p[4] ) )
    ll = ll .+ p10 .* ( (1-lambda) .* p[4] + lambda.*(1 .- p[1] .- p[2] .- p[3] ) )

    return -sum(log.(max.(ll,0)))    
end

Y,X = generate_data(200)
print(X,Y)
ll_obj([1 4 -3 2 3 2 -1];xs=X,ys=Y)

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

In [6]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X,ys=Y)
print(result.minimizer)
print(result.minimum)

UndefVarError: UndefVarError: X not defined

In [5]:
function monte_carlo()
    trials = 400
    params_200 = zeros((7,trials))
    params_1000 = zeros((7,trials))
    for i =1:trials
        Y_200, X_200 = generate_data(200)
        trial_200 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_200,ys=Y_200)
        params_200[:,i] = trial_200.minimizer
        
        Y_1000, X_1000 = generate_data(1000)
        trial_1000 = maximum_likelihood([1 4 -3 2 3 -2 -1.];xs=X_1000,ys=Y_1000)
        params_1000[:,i] = trial_1000.minimizer
    end
    return params_200,params_1000
end

results_200,results_1000 = monte_carlo()

([1.04422 1.73388 … 1.4266 0.965539; 5.14216 4.23158 … 3.6236 3.33035; … ; -2.19754 -2.04235 … -1.59631 -2.62597; -25.1336 -0.617818 … -24.2954 -1.24622], [1.09906 0.909652 … 1.53894 0.88531; 3.86996 3.84547 … 4.85064 4.08251; … ; -2.36363 -2.13283 … -1.81146 -1.82162; -2.04263 -3.05238 … -2.00537 -23.5485])

In [15]:
println(mean(results_200[7,:]))
lambda_200 = exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))
println(lambda_200)
var_200 = std(results_200[7,:]) *exp(mean(results_200[7,:]))/(1 + exp(mean(results_200[7,:])))^2

-9.456451128415672
7.817745048836213e-5


0.0010853947270296244

In [16]:
println(mean(results_1000[7,:]))
lambda_1000 = exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))
println(lambda_1000)
var_200 = std(results_1000[7,:]) *exp(mean(results_1000[7,:]))/(1 + exp(mean(results_1000[7,:])))^2

-7.367289431055386
0.0006311791574525345


0.006119240674747745

In [17]:
function print_results(params_200, params_1000)
       
    params_200[7,:] = exp.(params_200[7,:])./(1 .+ exp.(params_200[7,:]))
    params_1000[7,:] = exp.(params_1000[7,:])./(1 .+ exp.(params_1000[7,:]))
    
    println("\\begin{center}")
    println("\\begin{table}")
    println("\\normalsize")
    println("\\begin{tabular}{c|c|c|c|c|c}")
    print("& True & Mean & SD  &  Mean  & SD  \\\\\n")
    print("& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\\\\n")
    println("\\hline \\hline")
    param_names = ["\$ \\alpha_1 \$","\$ \\beta_1 \$", "\$ \\Delta_1 \$", 
        "\$ \\alpha_2 \$", "\$ \\beta_2 \$", "\$ \\Delta_2 \$","\$ \\lambda \$" ]
    true_param = ["1", "4", "-3", "2", "3", "-2", ".25"]
    for i=1:7
        print("$(param_names[i]) &")
        print("$(true_param[i]) &")
        print(" $(round(mean(params_200[i,:]), digits=4)) &")
        print(" $(round(std(params_200[i,:]), digits=3)) &")
        print(" $(round(mean(params_1000[i,:]), digits=4)) &")
        print(" $(round(std(params_1000[i,:]), digits=3)) \\\\\n")
        if (i ==3) | (i==6)
            println( "\\hline \\hline")
        else
            println( "\\hline")
        end
        
    end
    println("\\end{tabular}")
    println("\\end{table}")
    println("\\end{center}")
end

print_results(results_200[:,:], results_1000[:,:])

\begin{center}
\begin{table}
\normalsize
\begin{tabular}{c|c|c|c|c|c}
& True & Mean & SD  &  Mean  & SD  \\
& &  (200 obs) &  (200 obs) &  (1000 obs) &  (1000 obs)  \\
\hline \hline
$ \alpha_1 $ &1 & 1.1764 & 0.536 & 1.0412 & 0.224 \\
\hline
$ \beta_1 $ &4 & 4.3211 & 0.815 & 4.0567 & 0.28 \\
\hline
$ \Delta_1 $ &-3 & -3.2811 & 0.867 & -3.0593 & 0.349 \\
\hline \hline
$ \alpha_2 $ &2 & 2.0197 & 0.462 & 2.0189 & 0.211 \\
\hline
$ \beta_2 $ &3 & 3.1135 & 0.53 & 3.0438 & 0.209 \\
\hline
$ \Delta_2 $ &-2 & -2.0867 & 0.606 & -2.0502 & 0.252 \\
\hline \hline
$ \lambda $ &.25 & 0.2447 & 0.301 & 0.149 & 0.15 \\
\hline
\end{tabular}
\end{table}
\end{center}
